In [1]:
import random

import pyspark.sql.functions as F
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.window import Window

In [2]:
# Initialize a Spark Session
spark_conf = (
    SparkConf()
    .set("spark.driver.memory", "2g")
    .set("spark.jars.packages", "org.apache.hadoop:hadoop-client:3.3.4,io.delta:delta-spark_2.12:3.2.0")
    .set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .set("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
)
sc = SparkContext.getOrCreate(spark_conf)
spark = SparkSession(sc)

bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
25/04/29 21:02:22 WARN Utils: Your hostname, Zunia resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/04/29 21:02:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/zunia_linux/Project/studia/data-eng/lab3/.venv/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/zunia_linux/.ivy2/cache
The jars for the packages stored in: /home/zunia_linux/.ivy2/jars
org.apache.hadoop#hadoop-client added as a dependency
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-679f54a7-45d7-4232-b4da-6c82ab5299df;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-client;3.3.4 in central
	found org.apache.hadoop#hadoop-common;3.3.4 in central
	found org.apache.hadoop.thirdparty#hadoop-shaded-protobuf_3_7;1.1.1 in central
	found org.apache.hadoop#hadoop-annotations;3.3.4 in central
	found org.apache.hadoop.thirdparty#hadoop-shaded-guava;1.1.1 in central
	found com.google.guava#guava;27.0-jre in central
	found com.google.guava#failureaccess;1.0 in central
	found com.google.guava#listenablefuture;9999.0-empty-to-avoid-conflict-with-guava in central
	found com.google.code.findbugs#jsr305;3.0.2 in central
	found org.checkerframework#checker-qual;2.5.2 in central
	found com.goo

In [3]:
print(f"Hadoop version = {spark._jvm.org.apache.hadoop.util.VersionInfo.getVersion()}")
print(f"Spark version = {spark.version}")

Hadoop version = 3.3.4
Spark version = 3.5.0


In [4]:
sample_data = [
    (f'field1_{i}', i, i % 3)
    for i in range(1000)
]
sample_df = spark.createDataFrame(sample_data, ['field1', 'field2', 'field3'])
sample_df.write.format("delta").mode("overwrite").save("/home/zunia_linux/Project/studia/data-eng/lab3/sample_data")

25/04/29 21:02:33 WARN MemoryManager: Total allocation exceeds 95.00% (2,040,109,440 bytes) of heap memory
Scaling row group sizes to 95.00% for 16 writers


In [5]:
%%bash
mkdir -p /home/zunia_linux/Project/studia/data-eng/lab3/lakehouse/bronze
mkdir -p /home/zunia_linux/Project/studia/data-eng/lab3lakehouse/silver
mkdir -p /home/zunia_linux/Project/studia/data-eng/lab3lakehouse/gold

bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)


In [6]:
BRONZE_ROOT = "/home/zunia_linux/Project/studia/data-eng/lab3/lakehouse/bronze"
SILVER_ROOT = "/home/zunia_linux/Project/studia/data-eng/lab3/lakehouse/silver"
GOLD_ROOT = "/home/zunia_linux/Project/studia/data-eng/lab3/lakehouse/gold"

In [7]:
bronze_user = [
    dict(
        id=i,
        user_name=f"name{random.randint(1, 10)}",
        location=random.randint(1, 100),
    )
    for i in range(10_000)
]
section_ids = list(range(1, 10))
bronze_user_to_section = []
for user_id in range(10_000):
    section = random.sample(section_ids, random.randint(0, 5))
    if len(section) > 0:
        bronze_user_to_section += [dict(user_id=user_id, section_id=s) for s in section]

location_info = [
    dict(
        id=i,
        district_name=f"district{i % 10}",
        city_name=f"city{i % 30}"
    )
    for i in range(1, 101)
]

In [8]:
spark.createDataFrame(bronze_user).write.format("delta").mode("overwrite").save(f"{BRONZE_ROOT}/bronze_user")
spark.createDataFrame(bronze_user_to_section).write.format("delta").mode("overwrite").save(
    f"{BRONZE_ROOT}/bronze_user_to_section")
spark.createDataFrame(location_info).write.format("delta").mode("overwrite").save(f"{BRONZE_ROOT}/location_info")

In [9]:
users_df = spark.read.format("delta").load(f"{BRONZE_ROOT}/bronze_user")
user2section_df = spark.read.format("delta").load(f"{BRONZE_ROOT}/bronze_user_to_section")
location_df = spark.read.format("delta").load(f"{BRONZE_ROOT}/location_info")

In [10]:
print("Users")
users_df.show(5)
print("User to section")
user2section_df.show(5)
print("Location")
location_df.show(5)

Users


25/04/29 21:02:40 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---+--------+---------+
| id|location|user_name|
+---+--------+---------+
|625|       6|    name8|
|626|      66|    name8|
|627|      92|    name8|
|628|      42|    name5|
|629|      82|    name9|
+---+--------+---------+
only showing top 5 rows

User to section
+----------+-------+
|section_id|user_id|
+----------+-------+
|         8|   2817|
|         7|   2817|
|         6|   2817|
|         3|   2818|
|         7|   2818|
+----------+-------+
only showing top 5 rows

Location
+---------+-------------+---+
|city_name|district_name| id|
+---------+-------------+---+
|   city25|    district5| 55|
|   city26|    district6| 56|
|   city27|    district7| 57|
|   city28|    district8| 58|
|   city29|    district9| 59|
+---------+-------------+---+
only showing top 5 rows



In [11]:
pivoted_section = (
    user2section_df
    .groupBy('user_id')
    .pivot('section_id', [str(section_id) for section_id in section_ids])
    .agg(F.first('section_id'))
).select(
    F.col('user_id').alias('id'),
    *[
        F.col(str(section_id)).isNull().alias(f"attend_{section_id}") for section_id in section_ids
    ]
)

In [12]:
pivoted_section.show()

+----+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|  id|attend_1|attend_2|attend_3|attend_4|attend_5|attend_6|attend_7|attend_8|attend_9|
+----+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| 474|    true|    true|    true|    true|    true|    true|   false|    true|    true|
| 964|    true|    true|    true|    true|    true|   false|    true|    true|    true|
|9233|    true|    true|    true|    true|   false|   false|   false|    true|    true|
|4894|    true|    true|   false|    true|   false|    true|    true|   false|   false|
|5385|   false|   false|    true|    true|   false|    true|   false|   false|    true|
|5556|    true|    true|   false|    true|   false|   false|    true|    true|   false|
|5409|    true|    true|    true|    true|    true|    true|    true|   false|    true|
|3506|   false|   false|    true|    true|    true|    true|   false|   false|    true|
|3091|    true|   false|   false

In [13]:
silver_df = users_df.join(pivoted_section, 'id', 'left')
silver_df.show()

+---+--------+---------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| id|location|user_name|attend_1|attend_2|attend_3|attend_4|attend_5|attend_6|attend_7|attend_8|attend_9|
+---+--------+---------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|625|       6|    name8|   false|    true|    true|    true|    true|    true|    true|   false|    true|
|626|      66|    name8|   false|    true|    true|    true|   false|   false|   false|    true|   false|
|627|      92|    name8|   false|    true|    true|   false|    true|   false|   false|   false|    true|
|628|      42|    name5|    NULL|    NULL|    NULL|    NULL|    NULL|    NULL|    NULL|    NULL|    NULL|
|629|      82|    name9|    NULL|    NULL|    NULL|    NULL|    NULL|    NULL|    NULL|    NULL|    NULL|
|630|      53|    name5|   false|   false|    true|    true|   false|    true|   false|   false|    true|
|631|      29|    name1|    true|   false|   f

In [14]:
location_df = location_df.withColumnRenamed('id', 'location')
silver_df = silver_df.join(location_df, 'location', 'left')
silver_df.printSchema()

root
 |-- location: long (nullable = true)
 |-- id: long (nullable = true)
 |-- user_name: string (nullable = true)
 |-- attend_1: boolean (nullable = true)
 |-- attend_2: boolean (nullable = true)
 |-- attend_3: boolean (nullable = true)
 |-- attend_4: boolean (nullable = true)
 |-- attend_5: boolean (nullable = true)
 |-- attend_6: boolean (nullable = true)
 |-- attend_7: boolean (nullable = true)
 |-- attend_8: boolean (nullable = true)
 |-- attend_9: boolean (nullable = true)
 |-- city_name: string (nullable = true)
 |-- district_name: string (nullable = true)



In [15]:
silver_df.show()

+--------+---+---------+--------+--------+--------+--------+--------+--------+--------+--------+--------+---------+-------------+
|location| id|user_name|attend_1|attend_2|attend_3|attend_4|attend_5|attend_6|attend_7|attend_8|attend_9|city_name|district_name|
+--------+---+---------+--------+--------+--------+--------+--------+--------+--------+--------+--------+---------+-------------+
|       6|625|    name8|   false|    true|    true|    true|    true|    true|    true|   false|    true|    city6|    district6|
|      66|626|    name8|   false|    true|    true|    true|   false|   false|   false|    true|   false|    city6|    district6|
|      92|627|    name8|   false|    true|    true|   false|    true|   false|   false|   false|    true|    city2|    district2|
|      42|628|    name5|    NULL|    NULL|    NULL|    NULL|    NULL|    NULL|    NULL|    NULL|    NULL|   city12|    district2|
|      82|629|    name9|    NULL|    NULL|    NULL|    NULL|    NULL|    NULL|    NULL|   

In [16]:
silver_df.write.format("delta").mode("overwrite").save(f"{SILVER_ROOT}/users")

25/04/29 21:02:53 WARN MemoryManager: Total allocation exceeds 95.00% (2,040,109,440 bytes) of heap memory
Scaling row group sizes to 95.00% for 16 writers


In [17]:
silver_user = spark.read.format("delta").load(f"{SILVER_ROOT}/users")
report = silver_user.groupBy("city_name").agg(
    *[
        (F.sum(F.col(f"attend_{section_id}").cast("int")) / F.count("id")).alias(f"attend_level_{section_id}")
        for section_id in section_ids
    ]
)
report.show()

+---------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+
|city_name|    attend_level_1|    attend_level_2|    attend_level_3|    attend_level_4|    attend_level_5|    attend_level_6|     attend_level_7|    attend_level_8|    attend_level_9|
+---------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+
|    city7| 0.568922305764411|0.5664160401002506|0.5338345864661654| 0.543859649122807|0.5739348370927319|0.5463659147869674| 0.5639097744360902|0.5614035087719298|0.6090225563909775|
|   city24|0.5858585858585859|0.5084175084175084|0.5925925925925926|0.6060606060606061|0.5521885521885522|0.5387205387205387| 0.5858585858585859|0.5555555555555556|0.5959595959595959|
|   city15|0.5571428571428572|0.5892857142857143|               0.5|0.6107142857

In [18]:
report.write.format("delta").mode("overwrite").save(f"{GOLD_ROOT}/attends_per_city")
column = ','.join([f"'section_{section_id}', attend_level_{section_id}" for section_id in section_ids])
stack_expression =  f"stack(9, {column}) as (section, attend_level)"
print(stack_expression)
unpopular_per_city = report.selectExpr(
    "city_name",
    stack_expression
)

stack(9, 'section_1', attend_level_1,'section_2', attend_level_2,'section_3', attend_level_3,'section_4', attend_level_4,'section_5', attend_level_5,'section_6', attend_level_6,'section_7', attend_level_7,'section_8', attend_level_8,'section_9', attend_level_9) as (section, attend_level)


In [19]:
unpopular_per_city.show()

+---------+---------+------------------+
|city_name|  section|      attend_level|
+---------+---------+------------------+
|    city7|section_1| 0.568922305764411|
|    city7|section_2|0.5664160401002506|
|    city7|section_3|0.5338345864661654|
|    city7|section_4| 0.543859649122807|
|    city7|section_5|0.5739348370927319|
|    city7|section_6|0.5463659147869674|
|    city7|section_7|0.5639097744360902|
|    city7|section_8|0.5614035087719298|
|    city7|section_9|0.6090225563909775|
|   city24|section_1|0.5858585858585859|
|   city24|section_2|0.5084175084175084|
|   city24|section_3|0.5925925925925926|
|   city24|section_4|0.6060606060606061|
|   city24|section_5|0.5521885521885522|
|   city24|section_6|0.5387205387205387|
|   city24|section_7|0.5858585858585859|
|   city24|section_8|0.5555555555555556|
|   city24|section_9|0.5959595959595959|
|   city15|section_1|0.5571428571428572|
|   city15|section_2|0.5892857142857143|
+---------+---------+------------------+
only showing top

In [20]:
window_spec = Window.partitionBy('city_name').orderBy('attend_level')

unpopular_per_city_ranked = unpopular_per_city.withColumn('rank', F.row_number().over(window_spec))
unpopular_per_city_filtered = unpopular_per_city_ranked.filter(F.col('rank') == 1)
unpopular_per_city_filtered_result  = unpopular_per_city_filtered.drop('rank')
unpopular_per_city_filtered_result.show()

+---------+---------+-------------------+
|city_name|  section|       attend_level|
+---------+---------+-------------------+
|    city0|section_7|0.47735191637630664|
|    city1|section_1|  0.511002444987775|
|   city10|section_5| 0.5310173697270472|
|   city11|section_9|                0.5|
|   city12|section_7| 0.5366666666666666|
|   city13|section_1| 0.5295774647887324|
|   city14|section_9|  0.498371335504886|
|   city15|section_3|                0.5|
|   city16|section_2| 0.5457413249211357|
|   city17|section_3| 0.5409252669039146|
|   city18|section_5| 0.4856115107913669|
|   city19|section_3| 0.5244299674267101|
|    city2|section_2| 0.5449871465295629|
|   city20|section_1| 0.5292207792207793|
|   city21|section_9| 0.5057034220532319|
|   city22|section_9| 0.5084745762711864|
|   city23|section_9| 0.5114754098360655|
|   city24|section_2| 0.5084175084175084|
|   city25|section_4| 0.5340136054421769|
|   city26|section_7| 0.5292207792207793|
+---------+---------+-------------

In [21]:
unpopular_per_city_filtered_result.write.format("delta").mode("overwrite").save(f"{GOLD_ROOT}/unpopular_per_city")